In [10]:
import functions_class as fx
import classx as cl
import log_reg_functions as lrf
import loaddata as ld


import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import log_loss


# A ) 
Load data

In [7]:
x,y = ld.load_data(scaler='minmax')

xtrain,xtest,ytrain,ytest = train_test_split(x,y, test_size=0.25, random_state= 0, shuffle=True)

xtrain,xval,ytrain,yval = train_test_split(xtrain,ytrain, test_size=0.25, random_state= 0, shuffle=True)
# do this because want our test to be seperate, first splitting into test and train, and then splitting traindata 
#into training and validation

nx_train, ny_train = xtrain.shape
nx_test, ny_test = xtest.shape



# B) 
Egen logistic regression med gradient descent.


In [8]:
eta = 0.1 # learning rate
doplot = False
doprint = False
Niteration = 1000
beta = np.random.randn(x.shape[1],1)
costvec=[]
costvec_val=[]
loglvec=[]
xaxis=[]

%matplotlib qt
#plt.axis([0, Niteration, 0, 13])

start = time.time()
for iter in range(Niteration):
    
    sig = lrf.sigmoid(xtrain@beta)
    gradient = lrf.gradient_ols(xtrain,ytrain,sig)
    beta -= eta*gradient
    
    #Cost function
    cost = lrf.cost_log_ols(xtrain@beta,ytrain.T)
    cost_val = lrf.cost_log_ols(xval@beta,yval.T) # do this for testdata at the end. 
    #Log Loss function from sklearn
    if doprint:
        logloss=log_loss(ytrain, np.round(xtrain@beta), eps=1e-16, normalize=True)
        print('Cost', cost,'&','Log loss', logloss,'&','Cost test', cost_val)
    if doplot:
        costvec.append(cost.ravel())
        costvec_val.append(cost_val.ravel())
        xaxis.append(iter+1)
        plt.plot(xaxis, costvec, 'b')
        plt.plot(xaxis, costvec_val, 'r')
        plt.pause(1e-12)
plt.show()    
end = time.time()
print(end - start)
   
# et relevant plot er iterativt plot. 

5.3357391357421875


In [26]:
# making confusion matrix to check observed data with model predictions. 

predictions = xval@beta

sig_val = lrf.sigmoid(predictions)

 
for i in range(sig_val): 
    if sig_val >= 0.5:
        sig_val == 1
    if sig_val < 0.5:
        sig_val == 0


confusion_matrix = confusion_matrix(yval , sig_val)

TypeError: only integer scalar arrays can be converted to a scalar index

# Accuracy. 
Både egen kode og tester med scikit. 

In [249]:

activation =lrf.sigmoid(xtrain@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytrain)/len(activation),'% Training Accuracy')


activation =lrf.sigmoid(xtest@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytest)/len(activation),'% Test Accuracy')


80.60444444444444 % Training Accuracy
80.13333333333334 % Test Accuracy


# Egen logistisk regresjon med stokastisk gradient descent

In [246]:
eta = 0.1 # learning rate
doplot = False
doprint = False
Niteration = 1000
batch_size = 3000
batch_size_held_out = nx_train-batch_size

beta = np.random.randn(x.shape[1],1)
costvec=[]
costvec_test=[]
loglvec=[]
xaxis=[]

%matplotlib qt
#plt.axis([0, Niteration, 0, 13])

indexes = np.arange(nx_train)


start = time.time()
for iter in range(Niteration):
    datapoints = np.random.choice(indexes, size=batch_size, replace=False)
    batch_x = xtrain[datapoints,:]
    batch_y = ytrain[datapoints]
    
    batch_x_held_out = np.delete(xtrain, datapoints, axis=0)
    batch_y_held_out = np.delete(ytrain, datapoints).reshape([batch_size_held_out,1])
    
    sig = lrf.sigmoid(batch_x@beta)
    gradient = lrf.gradient_ols(batch_x,batch_y,sig)
    beta -= eta*gradient
    
    #Cost function
    cost = lrf.cost_log_ols(batch_x@beta,batch_y.T)
    cost_test = lrf.cost_log_ols(batch_x_held_out@beta,batch_y_held_out.T)
    #Log Loss function from sklearn
    if doprint:
        logloss=log_loss(batch_y, np.round(batch_x@beta), eps=1e-16, normalize=True)
        print('Cost', cost,'&','Log loss', logloss,'&','Cost test', cost_test)
    if doplot:
        costvec.append(cost.ravel())
        costvec_test.append(cost_test.ravel())
        xaxis.append(iter+1)
        plt.plot(xaxis, costvec, 'b')
        plt.plot(xaxis, costvec_test, 'r')
        plt.pause(1e-12)
plt.show()

end = time.time()
print(end - start)
 

16.83239507675171


In [247]:
activation =lrf.sigmoid(xtrain@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytrain)/len(activation),'% Training Accuracy')


activation =lrf.sigmoid(xtest@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytest)/len(activation),'% Test Accuracy')


80.60444444444444 % Training Accuracy
80.13333333333334 % Test Accuracy


In [248]:

model = LogisticRegression()
model.fit(xtrain, ytrain)
predicted_classes = model.predict(xtrain)
accuracy = accuracy_score(ytrain.flatten(),predicted_classes)
accuracy = accuracy * 100
parameters = model.coef_
log_loss(ytrain, predicted_classes)

print(accuracy, '% Training Accuracy')

predicted_classes = model.predict(xtest)
accuracy = accuracy_score(ytest.flatten(),predicted_classes)
accuracy = accuracy * 100
parameters = model.coef_
log_loss(ytest, predicted_classes)

print(accuracy, '% Test Accuracy')

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


82.19111111111111 % Training Accuracy
81.85333333333334 % Test Accuracy


# C) 
Neural Network. 

In [234]:
#self.X_data_full = X_data 
#self.Y_data_full = Y_data

n_inputs, n_features = x.shape

#self.epochs = epochs
#self.batch_size = batch_size
#self.iterations = self.n_inputs // self.batch_size
#self.eta = eta
#self.lmbd = lmbd


n_hidden_neurons=50  # velge andre verdier om vi vil
n_categories=2
epochs=100
batch_size=80
eta=0.0001
lmbd=0.1


hidden_weights = np.random.randn(n_features, n_hidden_neurons)
hidden_bias = np.zeros(n_hidden_neurons) + 0.01

output_weights = np.random.randn(n_hidden_neurons, n_categories)
output_bias = np.zeros(n_categories) + 0.01

In [235]:
print(hidden_weights.shape)
print(hidden_bias.shape)
print(output_weights.shape)
print(output_bias.shape)
print(x.shape)

(91, 50)
(50,)
(50, 2)
(2,)
(30000, 91)


In [236]:
z_h = np.matmul(x, hidden_weights) + hidden_bias
a_h = lrf.sigmoid(z_h)

z_o = np.matmul(a_h, output_weights) + output_bias

exp_term = np.exp(z_o)
probabilities = exp_term / np.sum(exp_term, axis=1, keepdims=True)

In [237]:
print(x.shape)
print(z_h.shape)
print(a_h.shape)
print(z_o.shape)
print(probabilities.shape)

(30000, 91)
(30000, 50)
(30000, 50)
(30000, 2)
(30000, 2)


In [238]:
probabilities

array([[0.00298381, 0.99701619],
       [0.69048859, 0.30951141],
       [0.00576687, 0.99423313],
       ...,
       [0.00690587, 0.99309413],
       [0.60833805, 0.39166195],
       [0.00249443, 0.99750557]])